In [ ]:
using Pkg; Pkg.add("Suppressor"); using Suppressor: @suppress;
@suppress Pkg.add(url="https://github.com/JuDO-dev/AirBorne.jl#dev");
@suppress Pkg.add(["JuMP","Dates","Plots","DataFrames", "DirectSearch","DotMaps","LinearAlgebra","Polynomials","SparseArrays","MathOptInterface","HiddenMarkovModels","Distributions"]);
bundle_id="Mark2"
cache_dir = joinpath(@__DIR__, "data", "cache")

In [ ]:
### NASDAQ Screener
using AirBorne.ETL.NASDAQ: screener
tickers_df = screener()
store_bundle(tickers_df; bundle_id="NASDAQscreener_"*bundle_id, archive=true, cache_dir=cache_dir)

In [ ]:
### Filter 2 most traded
using DataFrames: groupby, combine
using AirBorne.Utils: get_latest_N
topPerformers=2
bundle_id="Mark2"
filtered_df =tickers_df[[   x!="" ? parse(Int64, x)<2016 : false for x in tickers_df.ipoyear],["symbol","marketCap","sector","volume"]]
filtered_df[!,"volume"]=parse.(Int64,filtered_df[!,"volume"])
filtered_df[!,"marketCap"]=parse.(Float64,filtered_df[!,"marketCap"])
grouped_df = groupby(filtered_df,"sector")
f(sdf)= get_latest_N(sdf,:volume,topPerformers;rev=true)
result = combine(grouped_df,f)

In [ ]:
### Fetch data from yahoo finance
using AirBorne.ETL.Cache: store_bundle
using AirBorne.ETL.YFinance: get_interday_data
using AirBorne.ETL.Cache: store_bundle
using Dates: DateTime, datetime2unix
from = DateTime("2017-01-01"); to = DateTime("2022-01-01")
u_from = string(round(Int, datetime2unix(from)));
u_to = string(round(Int, datetime2unix(to)))
data = get_interday_data(result.symbol, u_from, u_to)
store_bundle(data; bundle_id=bundle_id, archive=true, cache_dir=cache_dir)
@info "Done!"
first(data,4)